In [1]:
from main import solve_sudoku

In [2]:
sudoku = (
    "000002000730050100010000530500040000342000000000860050900001000000430006000000800"
)

In [3]:
result = solve_sudoku(sudoku)

Unsolved Sudoku:

. . . | . . 2 | . . . 
7 3 . | . 5 . | 1 . . 
. 1 . | . . . | 5 3 . 
---------------------
5 . . | . 4 . | . . . 
3 4 2 | . . . | . . . 
. . . | 8 6 . | . 5 . 
---------------------
9 . . | . . 1 | . . . 
. . . | 4 3 . | . . 6 
. . . | . . . | 8 . . 

Solved in 4 iterations 

8 9 6 | 3 1 2 | 4 6 5 
7 3 6 | 9 5 8 | 1 2 8 
2 1 7 | 7 8 6 | 5 3 2 
---------------------
5 8 1 | 2 4 3 | 7 7 7 
3 4 2 | 1 1 5 | 6 8 5 
1 2 4 | 8 6 4 | 2 5 3 
---------------------
9 5 3 | 6 8 1 | 3 2 4 
8 7 9 | 4 3 5 | 9 2 6 
4 5 5 | 9 2 3 | 8 2 1 

